In [ ]:
# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
from skimage.io import imread
from skimage.transform import rescale, resize, downscale_local_mean, rotate
from skimage.util import random_noise

import matplotlib.pyplot as plt
#%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD


In [ ]:
# loading dataset
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')
sample_submission  = pd.read_csv('dataset/submission.csv')

In [ ]:
train.head()

In [ ]:
# loading training images
train_img = []
for img_name in tqdm(train['ID']):
    # defining the image path
    image_path = 'dataset/train/' + str(img_name) #+ '.png'
    # reading the image
    img2 = imread(image_path, as_gray=True)
    
    # converting the type of pixel to float 32
    
    img2 = resize(img2, (28, 28))
    # appending the image into the list
    img2 = img2.astype('float32')
    # normalizing the pixel values
    img2 /= 255.0
    
    #k = 0.1 # you could set any any real number
    #noise = np.ones_like(img2) * k * (img2.max() - img2.min())
    #noise[np.random.random(size=noise.shape) > 0.5] *= -1
    #img_noise = img2 + noise # new image with noise
    
    #image_rotated = rotate(img2, angle=180, resize=False)
    
    train_img.append(img2)


# converting the list to numpy array
train_x = np.array(train_img)
# defining the target
train_y = train['Label'].values
train_x.shape

In [ ]:
#visualizing images
i = 0
plt.figure(figsize=(5,5))
plt.subplot(221), plt.imshow(train_x[i], cmap='gray')
plt.subplot(222), plt.imshow(train_x[i+25], cmap='gray')
plt.subplot(223), plt.imshow(train_x[i+50], cmap='gray')
plt.subplot(224), plt.imshow(train_x[i+75], cmap='gray')

In [ ]:
# create validation set
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

In [ ]:
final_train_data = []
final_target_train = []
for i in tqdm(range(train_x.shape[0])):
    final_train_data.append(train_x[i])
    final_train_data.append(rotate(train_x[i], angle=45, mode = 'wrap'))
    final_train_data.append(np.fliplr(train_x[i]))
    final_train_data.append(np.flipud(train_x[i]))
    final_train_data.append(random_noise(train_x[i],var=0.00155**2))
    for j in range(5):
        final_target_train.append(train_y[i])

In [ ]:

final_train = np.array(final_train_data)
final_target_train = np.array(final_target_train)
len(final_target_train), len(final_train_data)

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=5,figsize=(20,20))
for i in range(5):
    ax[i].imshow(final_train[i+30])
    ax[i].axis('off')

In [ ]:
# converting training images into torch format
final_train = final_train.reshape(11375, 1, 28, 28)
final_train = torch.from_numpy(final_train).float()
#x_tensor = train_x(requires_grad=True)


# converting the target into torch format
final_target_train = final_target_train.astype(int);
final_target_train = torch.from_numpy(final_target_train)
y_tensor = final_target_train.long().clone().detach() 
# shape of training datacd 
final_train.shape, final_target_train.shape

In [ ]:
# converting validation images into torch format
val_x = val_x.reshape(253, 1, 28, 28)
val_x  = torch.from_numpy(val_x).float()

# converting the target into torch format
val_y = val_y.astype(int);
val_y = torch.from_numpy(val_y)
y_valsor = val_y.long().clone().detach() 

# shape of validation data
val_x.shape, val_y.shape

In [ ]:
class DARTS_V2(Module):   
    def __init__(self):
        super(DARTS_V2, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 10, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(10),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(10, 20, kernel_size=5, stride=1, padding=1),
            BatchNorm2d(20),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            
        )

        self.linear_layers = Sequential(
            Linear(720, 6)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
# defining the model
model = DARTS_V2()
# defining the optimizer
optimizer = SGD(model.parameters(), lr=0.025, momentum=0.9, weight_decay=0.0003)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

In [ ]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(final_train), Variable(y_tensor)
    # getting the validation set
    x_val, y_val = Variable(val_x), Variable(y_valsor)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)
        

In [ ]:
n_epochs = 500
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
    train(epoch)

In [ ]:
# plotting the training and validation loss
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.show()

In [ ]:
# prediction for training set
with torch.no_grad():
    output = model(final_train)
    
softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on training set
accuracy_score(final_target_train, predictions)

In [ ]:
print(train_y.grad)

In [ ]:
# prediction for validation set
with torch.no_grad():
    output = model(val_x)

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on validation set
accuracy_score(val_y, predictions)

In [ ]:
test_img = []
for img_name in tqdm(test['ID']):
    # defining the image path
    image_path = 'dataset/test/' + str(img_name) #+ '.png'
    # reading the image
    img2 = imread(image_path, as_gray=True)
    
    # converting the type of pixel to float 32
    
    img2 = resize(img2, (28, 28))
    # appending the image into the list
    img2 = img2.astype('float32')
    # normalizing the pixel values
    img2 /= 255.0
    
    #k = 0.1 # you could set any any real number
    #noise = np.ones_like(img2) * k * (img2.max() - img2.min())
    #noise[np.random.random(size=noise.shape) > 0.5] *= -1
    #img_noise = img2 + noise # new image with noise
    
    image_rotated = rotate(img2, angle=180, resize=False)
    

    test_img.append(image_rotated)

# converting the list to numpy array
test_x = np.array(test_img)
test_x.shape

# converting training images into torch format
test_x = test_x.reshape(10142, 1, 28, 28)
test_x  = torch.from_numpy(test_x)
test_x.shape

In [ ]:
test_x=test_x.float()

In [ ]:
# generating predictions for test set
with torch.no_grad():
    output = model(test_x)

softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

In [ ]:
# replacing the label with prediction
sample_submission['Label'] = predictions
sample_submission.head()

In [ ]:
# saving the file
sample_submission.to_csv('surya27.csv', index=False)